In [1]:
%set_env ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}%set_env LANGSMITH_TRACING=true%set_env LANGSMITH_API_KEY=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%set_env TAVILY_API_KEY=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%set_env OPENAI_API_KEY=${OPENAI_API_KEY}%set_env LANGCHAIN_API_KEY=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%set_env LANGCHAIN_TRACING_V2=true%set_env LANGCHAIN_PROJECT="Multi-agent Collaboration"

env: ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}
env: LANGSMITH_TRACING=true
env: LANGSMITH_API_KEY=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
env: TAVILY_API_KEY=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
env: OPENAI_API_KEY=${OPENAI_API_KEY}
env: LANGCHAIN_API_KEY=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
env: LANGCHAIN_TRACING_V2=true
env: LANGCHAIN_PROJECT="Multi-agent Collaboration"


In [2]:
import config# from langchain.chat_models import ChatOpenAIfrom langchain_openai import ChatOpenAIfrom langchain_anthropic import ChatAnthropicfrom langchain.schema import AIMessage, HumanMessage, SystemMessagedef ChatAnthropicSKT(model="anthropic/claude-3-5-sonnet-20240620"):    llm_api_key = config.CUSTOM_API_KEY"https://api.platform.a15t.com/v1"    # model = "anthropic/claude-3-5-sonnet-20240620"    model = ChatOpenAI(        temperature=0,          openai_api_key=llm_api_key,         openai_api_base=llm_api_url,         model=model        )    return modelllm = ChatAnthropicSKT()

/Users/yongwook/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import operator
from typing import Annotated, Sequence, TypedDict, List, Optional, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langgraph.graph import END, StateGraph, START
from langchain_core.tools import tool

class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: Optional[str]
    dataframe_json: str
    df_schema: Optional[str]
    dataframe_path: str
    sql_query: Optional[str]
    search_result: Optional[str]

from langchain.tools import StructuredTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

def create_agent(llm, tools: List[StructuredTool], system_message: str):
    """Create an agent."""

    tool_names = [tool.name for tool in tools]
    tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=tool_names)
    # prompt = prompt.partial(tool_descriptions=tool_descriptions)
    return prompt | llm.bind_tools(tools)

@tool
def check_schema(
    dataframe_json: Annotated[str, "dataframe in JSON format."], 
    dataframe_path: Annotated[str, "path to the dataframe."]
):    
    """
    Check the schema of a dataframe.
    
    Args:
        dataframe_json (str): The dataframe in JSON format.
        dataframe_path (str): The path to the dataframe.
    
    Returns:
        str: The schema of the dataframe.
    """

    print("----------------- dataframe_json:", dataframe_json)
    print("----------------- dataframe_path:", dataframe_path)

    try:

        # If it's a string, try to parse it as JSON
        try:
            if isinstance(dataframe_json, str):
                json_data = json.loads(dataframe_json)
            else:
                json_data = dataframe_json
            
            # Create a DataFrame
            df = pd.DataFrame(json_data)
        except:
            df = pd.read_csv(dataframe_path)
        
        # Get the schema
        schema = df.dtypes.to_string()
        return f"DataFrame schema:\n{schema}"
    except json.JSONDecodeError as e:
        error_msg = f"Error decoding JSON string: {str(e)}"
        return error_msg
    except Exception as e:
        error_msg = f"Error checking schema: {str(e)}"
        return error_msg

# check_schema_tool = StructuredTool.from_function(
#     func=check_schema,
#     name="check_schema",
#     description="Check the schema of a pandas DataFrame"
# )

from langchain_core.messages import AIMessage, ToolMessage

# Helper function to create a node for a given agent
def agent_node(state, agent, name):

    print(name, state)

    result = agent.invoke(state)

    print(result)

    output = result["output"]

    new_state = state.copy()
    new_state["messages"] = state["messages"] + [HumanMessage(content=output, name=name)]
    
    if name == "Schema_Analyst":
        new_state["df_schema"] = output if "DataFrame schema:" in output else state['df_schema']
    elif name == "SQL_Expert":
        new_state["sql_query"] = output
    
    return new_state

import functools

schema_agent = create_agent(llm, [check_schema], "You are a data analyst specializing in understanding data schemas.")

schema_node = functools.partial(agent_node, agent=schema_agent, name="Schema_Analyst")

from langgraph.prebuilt import ToolNode

tools = [check_schema]
tool_node = ToolNode(tools)

# Either agent can decide to end
from typing import Literal

def router(state) -> Literal["call_tool", "__end__", "continue"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "__end__"
    return "continue"

workflow = StateGraph(AgentState)

workflow.add_node("Schema_Analyst", schema_node)
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "Schema_Analyst",
    router,
    {"continue": END, "call_tool": "call_tool", "__end__": END},
)

workflow.add_edge(START, "Schema_Analyst")
graph = workflow.compile()

import pandas as pd

df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'San Francisco', 'Los Angeles']
})

initial_state = AgentState(
    messages=[HumanMessage(content="find the schema of the dataframe", name="Human")],
    team_members=[
        "Schema_Analyst",
        # "SQL_Expert", 
        # "Data_Scientist"
                  ],
    next=None,
    dataframe_json=df.to_json(orient='records'),
    dataframe_path="data/example.csv",
    df_schema=None,
    sql_query=None,
    search_result=None
)

i = 0
for output in graph.stream(initial_state):
    print("Workflow output:", output)
    if i > 20:
        break
    i += 1


ModuleNotFoundError: No module named 'pydantic.v1'